In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug  9 20:02:23 2020
@author: MUSTAFAAKTAS

Modified Aug  2022
@author: cmmarellano
"""

import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import geopandas as gpd 
import geoio
import json

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath("C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\")
dataset_dir = os.path.abspath("C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\")

In [3]:
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [4]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [5]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\mask_rcnn_spacenet_0151\\mask_rcnn_spacenet_0151.h5")
print(COCO_MODEL_PATH)
print('- - loaded')

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
#     utils.download_trained_weights(COCO_MODEL_PATH)
    print('Specify COCO_MODEL_PATH !')
    

C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\mask_rcnn_spacenet_0151\mask_rcnn_spacenet_0151.h5
- - loaded


In [6]:
# define data locations for images and annotations
images_dir = os.path.join(dataset_dir, "PS-RGB_select_png\\")
annotations_dir = os.path.join(dataset_dir, "building_labels_select_geojson\\")
        

In [7]:
# #start=1, end=400
# start=1
# end=400
# counters = 1
# for filename in os.listdir(images_dir)[start:end]:
#     print(counters)
#     print(filename)        
#     id_base  = os.path.basename(filename)
#     finder= 'PS-RGB_img'
#     id_start = filename.find(finder)+ len(finder)
#     id_end = filename.find(".png")
#     image_id = filename[id_start:id_end]
#     image_dir = os.path.join(images_dir,str(filename))
#     ann_path  = os.path.join(annotations_dir,"buildings_AOI_5_Khartoum_img"+str(image_id)+".geojson")   #
#     print("image id : ",image_id)
#     print("image dir : ",image_dir)
#     print("ann path : ",ann_path)
#     counters = counters+1

# fname = os.path.basename(list_png[DosyaNumarasi])
#     finder = 'PS-RGB_img'
#     start = fname.find(finder)+6
#     end = fname.find('.png')
#     img_name = fname[start:end]


In [8]:
image_dir = "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\PS-RGB_select_png\\PS-RGB_img1003.png"
im = Image.open(image_dir)
np.asarray(im)

array([[[140, 117, 117],
        [157, 133, 133],
        [156, 132, 133],
        ...,
        [ 45,  34,  58],
        [ 29,  19,  42],
        [ 44,  33,  56]],

       [[128, 105, 105],
        [146, 122, 120],
        [149, 125, 125],
        ...,
        [ 48,  37,  59],
        [ 44,  33,  56],
        [ 38,  28,  52]],

       [[132, 109, 108],
        [135, 113, 113],
        [141, 118, 117],
        ...,
        [ 49,  38,  61],
        [ 39,  29,  52],
        [ 42,  32,  56]],

       ...,

       [[130,  95,  84],
        [117,  83,  72],
        [ 61,  30,  19],
        ...,
        [181, 150, 133],
        [177, 146, 128],
        [172, 141, 124]],

       [[138, 100,  91],
        [115,  80,  69],
        [ 82,  49,  38],
        ...,
        [159, 132, 117],
        [157, 130, 114],
        [164, 135, 119]],

       [[148, 110, 100],
        [126,  89,  78],
        [123,  86,  77],
        ...,
        [143, 120, 105],
        [153, 128, 113],
        [189, 161, 147]]

In [9]:
class SpaceNetConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "SpaceNet"
    BACKBONE = "resnet50"
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1 building

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 640
    IMAGE_MAX_DIM = 640

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    RPN_ANCHOR_RATIOS = [0.25, 1, 4]

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 10    #32

    USE_MINI_MASK = True
    
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 500

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 50
    
    MAX_GT_INSTANCES=250
    DETECTION_MAX_INSTANCES=350
    
config = SpaceNetConfig()
config.display()





Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        350
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  640
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

In [10]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def fill_between(polygon):
    """
    Returns: a bool array
    """
    img = Image.new('1', (650, 650), False)
    ImageDraw.Draw(img).polygon(polygon, outline=True, fill=True)
    mask = np.array(img)
    return mask

In [11]:


class SpaceNetDataset(utils.Dataset):
    """Generates the shapes synthetic dataset. The dataset consists of simple
    shapes (triangles, squares, circles) placed randomly on a blank surface.
    The images are generated on the fly. No file access required.
    """

    
    def load_dataset(self, dataset_dir=dataset_dir, start=0, end=10):   # 1-400
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        
        self.start=start
        self.end=end
        
        # Add classes
        self.add_class("SpaceNetDataset", 1, "building")

        # define data locations for images and annotations
        images_dir = os.path.join(dataset_dir, "PS-RGB_select_png\\")
        annotations_dir = os.path.join(dataset_dir, "building_labels_select_geojson\\")
        
        # Iterate through all files in the folder to 
        #add class, images and annotaions
        
        image_id_list =[]
        counters = 1
        for filename in os.listdir(images_dir)[start:end]:
            print(counters)
            print(filename)        
            id_base  = os.path.basename(filename)
            finder= 'PS-RGB_img'
            id_start = filename.find(finder)+ len(finder)
            id_end = filename.find(".png")
            image_id = filename[id_start:id_end]
            image_dir = os.path.join(images_dir,str(filename))
            ann_path  = os.path.join(annotations_dir,"label_img"+str(image_id)+".geojson")  
            print("image id : ",image_id)
            print("image dir : ",image_dir)
            print("ann path : ",ann_path)
            
            self.add_image('SpaceNetDataset', image_id=image_id, path=image_dir, annotation=ann_path)
            counters = counters+1
            image_id_list.append(image_id)
        print("\nimage ids loaded: ", image_id_list)
        
        self.image_id_list = image_id_list
    
        

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
            Typically this function loads the image from a file, but
            in this case it generates the image on the fly from the
            specs in image_info.
        """
        self.load_dataset()
        cont_id = image_id
        image_id = self.image_id_list[cont_id]
        
        # define data locations for images and annotations
        images_dir = os.path.join(dataset_dir, "PS-RGB_select_png\\")
        annotations_dir = os.path.join(dataset_dir, "building_labels_select_geojson\\")

        # Iterate through all files in the folder to 
        #add class, images and annotaions   

        image_dir = os.path.join(dataset_dir+"\\PS-RGB_select_png\\PS-RGB_img"+str(image_id)+".png")
        im = Image.open(image_dir)
        return np.asarray(im)


        
    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        
        self.load_dataset()
        cont_id = image_id
        image_id = self.image_id_list[cont_id]
        
        
        masks = np.zeros((650,650))
        bldgmask_path = "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\mask_for_training\\"
        ResimPATH = bldgmask_path+"mask_img"+str(image_id)+'.png'
        RGBTIFResmi = geoio.GeoImage(ResimPATH)
        
        with open(dataset_dir+"\\building_labels_select_geojson\\label_img"+str(image_id)+".geojson") as f:
            data = json.load(f)
            allBuildings = data['features']
              
            for building in allBuildings:   
                veri = building['geometry']['coordinates'][0]
                
                tip = str(building['geometry']['type'])
                coordinates = list() 
                if tip == ('Point'):
                    continue
                    
                elif tip == ('MultiPolygon'):
                    
                    if isinstance(veri,float): continue
                        
                    kucukBinalar = (building['geometry']['coordinates'])
                    for b in range(len(kucukBinalar)):  
                        veri = kucukBinalar[b][0]
                        for i in veri:
                            xPixel, yPixel = RGBTIFResmi.proj_to_raster(i[0], i[1])
                            xPixel = 649 if xPixel > 649 else xPixel
                            yPixel = 649 if yPixel > 649 else yPixel
                            coordinates.append((xPixel,yPixel)) 
                else:
                    if isinstance(veri,float): continue
    
                    for i in veri:
                        xPixel, yPixel = RGBTIFResmi.proj_to_raster(i[0], i[1])
                        xPixel = 649 if xPixel > 649 else xPixel
                        yPixel = 649 if yPixel > 649 else yPixel
                        coordinates.append((xPixel,yPixel))

                maske = fill_between(coordinates)
                masks = np.dstack((masks,maske))

                
        if masks.shape != (650,650):
            masks = masks[:,:,1:]
            class_ids = np.asarray([1]*masks.shape[2])
        else:
            class_ids=np.ones((1))
            masks = masks.reshape((650,650,1))
        return masks.astype(np.bool), class_ids.astype(np.int32)
    



In [12]:
# Training dataset
dataset_train = SpaceNetDataset()
dataset_train.load_dataset(dataset_dir,0,12) 
dataset_train.prepare()

1
PS-RGB_img1003.png
image id :  1003
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1003.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1003.geojson
2
PS-RGB_img1004.png
image id :  1004
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1004.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1004.geojson
3
PS-RGB_img1113.png
image id :  1113
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1113.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1113.geojson
4
PS-RGB_img1114.png
image id :  1114
image dir :  C:\Users\carellano.LEG

In [13]:
# Validation dataset
dataset_val = SpaceNetDataset()
dataset_val.load_dataset(dataset_dir,12,16)
dataset_val.prepare()

1
PS-RGB_img1348.png
image id :  1348
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1348.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1348.geojson
2
PS-RGB_img1452.png
image id :  1452
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1452.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1452.geojson
3
PS-RGB_img1453.png
image id :  1453
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1453.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1453.geojson
4
PS-RGB_img1454.png
image id :  1454
image dir :  C:\Users\carellano.LEG

In [14]:
"""
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids,4)
for image_id in image_ids:
    print(image_id)
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
"""
"""
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
# Which weights to start with?
init_with = "last"  # imagenet, coco, or last
if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    #model.load_weights(model.find_last(), by_name=True)
    model.load_weights(model.find_last(), by_name=True)
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=200, 
            layers='all')
"""

'\n# Create model in training mode\nmodel = modellib.MaskRCNN(mode="training", config=config,\n                          model_dir=MODEL_DIR)\n# Which weights to start with?\ninit_with = "last"  # imagenet, coco, or last\nif init_with == "imagenet":\n    model.load_weights(model.get_imagenet_weights(), by_name=True)\nelif init_with == "coco":\n    # Load weights trained on MS COCO, but skip layers that\n    # are different due to the different number of classes\n    # See README for instructions to download the COCO weights\n    model.load_weights(COCO_MODEL_PATH, by_name=True,\n                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", \n                                "mrcnn_bbox", "mrcnn_mask"])\nelif init_with == "last":\n    # Load the last model you trained and continue training\n    #model.load_weights(model.find_last(), by_name=True)\n    model.load_weights(model.find_last(), by_name=True)\n# Train the head branches\n# Passing layers="heads" freezes all layers excep

In [15]:
#########################################################DETECTION

class InferenceConfig(SpaceNetConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()


In [16]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)


Instructions for updating:
Use fn_output_signature instead


In [17]:
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path =  COCO_MODEL_PATH   #model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights from  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\mask_rcnn_spacenet_0151\mask_rcnn_spacenet_0151.h5


In [28]:
# #extra
# import tensorflow as tf
# new_model_path = "C:\\Users\\carellano.LEGION5PRO-O3OB\\Documents\\IP AppDev\\Final_Project\\SpaceNet\\transferlearning\\"
# # tf.saved_model.save(model, new_model_path)

# modeler = model
# modeler.save("new_model_path")

In [29]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)


In [30]:
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =modellib.load_image_gt(dataset_val, inference_config, image_id) #use_mini_mask=False'



1
PS-RGB_img1003.png
image id :  1003
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1003.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1003.geojson
2
PS-RGB_img1004.png
image id :  1004
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1004.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1004.geojson
3
PS-RGB_img1113.png
image id :  1113
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1113.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1113.geojson
4
PS-RGB_img1114.png
image id :  1114
image dir :  C:\Users\carellano.LEG

C:\Users\carellano.LEGION5PRO-O3OB\AppData\Local\Temp\ipykernel_21248\2281089392.py:140: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return masks.astype(np.bool), class_ids.astype(np.int32)


In [21]:
log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

original_image           shape: (640, 640, 3)         min:    0.00000  max:  255.00000  uint8
image_meta               shape: (14,)                 min:    0.00000  max:  650.00000  float64
gt_class_id              shape: (0,)                  min:             max:             int32
gt_bbox                  shape: (0, 4)                min:             max:             int32
gt_mask                  shape: (56, 56, 0)           min:             max:             bool


In [19]:

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

results = model.detect([original_image], verbose=1)
r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())



C:\ProgramData\Miniconda3\envs\py39_tf\lib\site-packages\mask_rcnn-2.1-py3.9.egg\mrcnn\visualize.py:167: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
C:\Users\carellano.LEGION5PRO-O3OB\AppData\Roaming\Python\Python39\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,



*** No instances to display *** 

Processing 1 images
image                    shape: (640, 640, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 640, 640, 3)      min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  640.00000  int32
anchors                  shape: (1, 102300, 4)        min:   -0.20031  max:    1.10016  float32


In [32]:

# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 6)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id) #, use_mini_mask=False
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))


1
PS-RGB_img1003.png
image id :  1003
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1003.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1003.geojson
2
PS-RGB_img1004.png
image id :  1004
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1004.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1004.geojson
3
PS-RGB_img1113.png
image id :  1113
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1113.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1113.geojson
4
PS-RGB_img1114.png
image id :  1114
image dir :  C:\Users\carellano.LEG

C:\Users\carellano.LEGION5PRO-O3OB\AppData\Local\Temp\ipykernel_21248\2281089392.py:140: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return masks.astype(np.bool), class_ids.astype(np.int32)
C:\ProgramData\Miniconda3\envs\py39_tf\lib\site-packages\mask_rcnn-2.1-py3.9.egg\mrcnn\utils.py:734: RuntimeWarning: invalid value encountered in divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


1
PS-RGB_img1003.png
image id :  1003
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1003.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1003.geojson
2
PS-RGB_img1004.png
image id :  1004
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1004.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1004.geojson
3
PS-RGB_img1113.png
image id :  1113
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1113.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1113.geojson
4
PS-RGB_img1114.png
image id :  1114
image dir :  C:\Users\carellano.LEG

C:\Users\carellano.LEGION5PRO-O3OB\AppData\Local\Temp\ipykernel_21248\2281089392.py:140: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return masks.astype(np.bool), class_ids.astype(np.int32)
C:\ProgramData\Miniconda3\envs\py39_tf\lib\site-packages\mask_rcnn-2.1-py3.9.egg\mrcnn\utils.py:734: RuntimeWarning: invalid value encountered in divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


1
PS-RGB_img1003.png
image id :  1003
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1003.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1003.geojson
2
PS-RGB_img1004.png
image id :  1004
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1004.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1004.geojson
3
PS-RGB_img1113.png
image id :  1113
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1113.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1113.geojson
4
PS-RGB_img1114.png
image id :  1114
image dir :  C:\Users\carellano.LEG

...

C:\Users\carellano.LEGION5PRO-O3OB\AppData\Local\Temp\ipykernel_21248\2281089392.py:140: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return masks.astype(np.bool), class_ids.astype(np.int32)
C:\ProgramData\Miniconda3\envs\py39_tf\lib\site-packages\mask_rcnn-2.1-py3.9.egg\mrcnn\utils.py:734: RuntimeWarning: invalid value encountered in divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


1
PS-RGB_img1003.png
image id :  1003
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1003.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1003.geojson
2
PS-RGB_img1004.png
image id :  1004
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1004.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1004.geojson
3
PS-RGB_img1113.png
image id :  1113
image dir :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\PS-RGB_select_png\PS-RGB_img1113.png
ann path :  C:\Users\carellano.LEGION5PRO-O3OB\Documents\IP AppDev\Final_Project\SpaceNet\building_labels_select_geojson\label_img1113.geojson
4
PS-RGB_img1114.png
image id :  1114
image dir :  C:\Users\carellano.LEG

C:\Users\carellano.LEGION5PRO-O3OB\AppData\Local\Temp\ipykernel_21248\2281089392.py:140: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return masks.astype(np.bool), class_ids.astype(np.int32)


mAP:  nan


C:\ProgramData\Miniconda3\envs\py39_tf\lib\site-packages\mask_rcnn-2.1-py3.9.egg\mrcnn\utils.py:734: RuntimeWarning: invalid value encountered in divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)
